Table 4.1.1: Treatment exit reasons of all young people exiting treatment 2016-17

In [1]:
from gssutils import *

if is_interactive():
    import requests
    from cachecontrol import CacheControl
    from cachecontrol.caches.file_cache import FileCache
    from cachecontrol.heuristics import LastModified
    from pathlib import Path

    session = CacheControl(requests.Session(),
                           cache=FileCache('.cache'),
                           heuristic=LastModified())

    sourceFolder = Path('in')
    sourceFolder.mkdir(exist_ok=True)

    inputURL = 'https://www.gov.uk/government/uploads/system/uploads/attachment_data/file/664944/'\
                    'Young-people-statistics-data-tables-from-the-national-drug-treatment-monitoring-system-2016-2017.xls'
    inputFile = sourceFolder / 'Young-people-statistics-data-tables-from-the-national-drug-treatment-monitoring-system-2016-2017.xls'
    response = session.get(inputURL)
    with open(inputFile, 'wb') as f:
      f.write(response.content)    

In [2]:
tab = loadxlstabs(inputFile, sheetids='4.1.1 Service Exits')[0]

Loading in\Young-people-statistics-data-tables-from-the-national-drug-treatment-monitoring-system-2016-2017.xls which has size 281600 bytes
Table names: ['4.1.1 Service Exits']


In [3]:
observations = tab.excel_ref('B4').expand(DOWN).expand(RIGHT).is_not_blank()
observations

{<C9 0.0>, <B8 105.0>, <C5 0.11>, <B4 8842.0>, <C10 1.0>, <B5 1178.0>, <B7 197.0>, <C6 0.04>, <C8 0.01>, <C7 0.02>, <B9 43.0>, <B6 469.0>, <B10 10834.0>, <C4 0.82>}

In [4]:
setting = tab.excel_ref('A4').expand(DOWN).is_not_blank() 
setting

{<A4 'Completed'>, <A10 'Total'>, <A7 'Treatment declined by client'>, <A5 'Dropped out / left'>, <A8 'Other'>, <A9 'Prison'>, <A6 'Referred on'>}

In [5]:
mt = tab.excel_ref('B3').expand(RIGHT).is_not_blank() 
mt

{<B3 'n'>, <C3 '%'>}

In [6]:
Dimensions = [
            HDim(mt,'Measure Type',DIRECTLY,ABOVE),
            HDim(setting,'Clients in treatment',DIRECTLY, LEFT),
            HDimConst('Unit','People')            
            ]

In [7]:
c1 = ConversionSegment(observations, Dimensions, processTIMEUNIT=True)
# if is_interactive():
#     savepreviewhtml(c1)

In [8]:
new_table = c1.topandas()
new_table

,OBS,Measure Type,Clients in treatment,Unit
0,8842.00,n,Completed,People
1,0.82,%,Completed,People
2,1178.00,n,Dropped out / left,People
3,0.11,%,Dropped out / left,People
4,469.00,n,Referred on,People
5,0.04,%,Referred on,People
6,197.00,n,Treatment declined by client,People
7,0.02,%,Treatment declined by client,People
8,105.00,n,Other,People
9,0.01,%,Other,People


In [9]:
new_table.columns = ['Value' if x=='OBS' else x for x in new_table.columns]

In [10]:
new_table.head()

,Value,Measure Type,Clients in treatment,Unit
0,8842.00,n,Completed,People
1,0.82,%,Completed,People
2,1178.00,n,Dropped out / left,People
3,0.11,%,Dropped out / left,People
4,469.00,n,Referred on,People


In [11]:
new_table['Measure Type'] = new_table['Measure Type'].map(
    lambda x: {
        'n' : 'Count', 
        '%' : 'Percentage',
        }.get(x, x))

In [12]:
new_table.tail()

,Value,Measure Type,Clients in treatment,Unit
9,0.01,Percentage,Other,People
10,43.00,Count,Prison,People
11,0.00,Percentage,Prison,People
12,10834.00,Count,Total,People
13,1.00,Percentage,Total,People


In [13]:
new_table.dtypes

Value                   float64
Measure Type             object
Clients in treatment     object
Unit                     object
dtype: object

In [14]:
new_table['Value'] = new_table['Value'].astype(str)

In [15]:
new_table.head(3)

,Value,Measure Type,Clients in treatment,Unit
0,8842.0,Count,Completed,People
1,0.82,Percentage,Completed,People
2,1178.0,Count,Dropped out / left,People


In [16]:
new_table['Basis of treatment'] =  'Treatment exit reasons' 

In [17]:
new_table['Clients in treatment'] = new_table['Clients in treatment'].map(
    lambda x: {
        'Total' : 'All reasons', 
        }.get(x, x))

In [18]:
new_table['Period'] = '2016-17'
new_table['Substance'] = 'All'
new_table = new_table[['Period','Basis of treatment','Substance','Clients in treatment','Measure Type','Value','Unit']]

In [19]:
if is_interactive():
    SubstancetinationFolder = Path('out')
    SubstancetinationFolder.mkdir(exist_ok=True, parents=True)
    new_table.to_csv(SubstancetinationFolder / ('table4.1.1.csv'), index = False)

In [20]:
new_table.tail()

,Period,Basis of treatment,Substance,Clients in treatment,Measure Type,Value,Unit
9,2016-17,Treatment exit reasons,All,Other,Percentage,0.01,People
10,2016-17,Treatment exit reasons,All,Prison,Count,43.0,People
11,2016-17,Treatment exit reasons,All,Prison,Percentage,0.0,People
12,2016-17,Treatment exit reasons,All,All reasons,Count,10834.0,People
13,2016-17,Treatment exit reasons,All,All reasons,Percentage,1.0,People
